In [3]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

Using TensorFlow backend.


In [4]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [5]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [6]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [7]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [8]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [9]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=pd.rolling_mean(df_dict[sku]['Sales'], window=7).shift(1) 

In [10]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [11]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [31]:
def RF_model(X_train, X_test, y_train, y_test):
    tree=RandomForestRegressor( random_state=2)
    tree.fit(X_train,y_train)
    pred=tree.predict(X_test)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def NN_model(X_train, X_test, y_train, y_test):
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], activation='linear'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(40, activation='linear'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
    model.fit(X_train, y_train, epochs=1, batch_size=10, verbose=0)
    prediction=model.predict(X_test, verbose=0)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'NN'})
    return pred

def GB_model(X_train, X_test, y_train, y_test):
    params = {'n_estimators': 500, 'max_depth': 40, 'min_samples_split': 20,
          'learning_rate': 0.01, 'loss': 'ls'}
    model = ensemble.GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_model(X_train, X_test, y_train, y_test):
    model=xgb.XGBRegressor( learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1,gamma=0, subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27  )
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [32]:
train_size=800
X_train={}
X_test={}
y_train={}
y_test={}

for sku in list(df_dict.keys()):
    X_train[sku],X_test[sku],y_train[sku],y_test[sku] =  X[sku][0:train_size], X[sku][train_size:len(X[sku])], y[sku][0:train_size], y[sku][train_size:len(X[sku])]

In [33]:
# Fit Models
rf_models={};
for sku in list(df_dict.keys()):
    rf_models[sku]=RF_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

nn_models={};
for sku in list(df_dict.keys()):
    nn_models[sku]=NN_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

gb_models={};
for sku in list(df_dict.keys()):
    gb_models[sku]=GB_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )
    
xg_models={};
for sku in list(df_dict.keys()):
    xg_models[sku]=XG_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

In [34]:
all_pred={}
for sku in list(df_dict.keys()):
    y_test[sku].reset_index(drop=True, inplace=True)
    all_pred[sku]=pd.concat([rf_models[sku],nn_models[sku],gb_models[sku],xg_models[sku],y_test[sku]],axis=1)
    

In [23]:
def MAE_score(y_true,pred1, pred2,pred3,pred4):
    mae1 = mean_absolute_error(y_true, pred1)
    mae2 = mean_absolute_error(y_true, pred2)
    mae3 = mean_absolute_error(y_true, pred3)
    mae4 = mean_absolute_error(y_true, pred4)
    return [mae1.round(),mae2.round(),mae3.round(),mae4.round()]
MAE_score(y_test[1],rf_models[1],nn_models[1],gb_models[1],xg_models[1])

[322.0, 1833.0, 340.0, 375.0]

In [37]:
mae=[]
for sku in list(df_dict.keys()):
    mae.append(MAE_score(all_pred[sku]['Sales'],all_pred[sku]['RF'] ,all_pred[sku]['NN'],all_pred[sku]['GB'],all_pred[sku]['XG'] ) )
mae=pd.DataFrame(mae)
mae=mae.rename(columns = {0:'RF',1:'NN',2:'GB',3:'XG'})

best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae


,RF,NN,GB,XG,Best
1,322.0,1327.0,340.0,375.0,RF
2,374.0,1458.0,381.0,383.0,RF
3,495.0,2118.0,533.0,568.0,RF
4,714.0,2820.0,711.0,745.0,GB
5,302.0,1302.0,316.0,332.0,RF
6,329.0,1294.0,333.0,343.0,RF
7,643.0,2558.0,603.0,690.0,GB
8,518.0,1950.0,532.0,558.0,RF
9,641.0,2442.0,612.0,619.0,GB
10,431.0,1635.0,435.0,457.0,RF


In [45]:
def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":
            tree=RandomForestRegressor( random_state=2)
            tree.fit(X[sku],y[sku])
            pred=tree.predict(X[sku])
            pred=pd.DataFrame(pred)
            pred.reset_index(drop=True, inplace=True)
            pred=pred.rename(columns = {0:'RF'})
            d[sku]=pred
            
        
        elif scores.loc[sku]["Best"]=="NN":
            model = Sequential()
            model.add(Dense(20, input_dim=X_train.shape[1], activation='linear'))
            model.add(Dense(20, activation='relu'))
            model.add(Dense(40, activation='linear'))
            model.add(Dense(10, activation='relu'))
            model.add(Dense(1, activation='linear'))
            model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
            model.fit(X[sku], y[sku], epochs=1, batch_size=10, verbose=0)
            prediction=model.predict(X_test, verbose=0)
            pred=pd.DataFrame(prediction)
            pred.reset_index(drop=True, inplace=True)
            pred=pred.rename(columns = {0:'NN'})
            d[sku]=pred
                    
            
        elif scores.loc[sku]["Best"]=="GB":
            params = {'n_estimators': 500, 'max_depth': 40, 'min_samples_split': 20,
          'learning_rate': 0.01, 'loss': 'ls'}
            model = ensemble.GradientBoostingRegressor(**params)
            model.fit(X[sku], y[sku])
            prediction=model.predict(X[sku])
            pred=pd.DataFrame(prediction)
            pred.reset_index(drop=True, inplace=True)
            pred=pred.rename(columns = {0:'GB'})
            d[sku]=pred  
            
        elif scores.loc[sku]["Best"]=="XG":
            model=xgb.XGBRegressor( learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1,gamma=0,
                                   subsample=0.8, colsample_bytree=0.8,nthread=4,scale_pos_weight=1,seed=27  )
            model.fit(X[sku], y[sku])
            prediction=model.predict(X[sku])
            pred=pd.DataFrame(prediction)
            pred.reset_index(drop=True, inplace=True)
            pred=pred.rename(columns = {0:'XG'})
      
    return d      

In [46]:
bm_dict=best_model(mae,X,y)

In [47]:
sku_pred=pd.concat([ bm_dict[1],bm_dict[2],bm_dict[3],bm_dict[4],bm_dict[5],bm_dict[6],bm_dict[7],bm_dict[8],bm_dict[9],bm_dict[10],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred.shape

(935, 11)

In [44]:
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,5684.4,6190.1,10332.7,10026.538214,5738.2,7145.6,10119.290515,6346.8,6191.985156,5568.9,73344.513886
1,5373.7,6552.9,8168.0,8867.965115,5864.3,6759.3,7198.112451,5334.5,4813.206130,5609.5,64541.483696
2,4946.9,5559.8,7790.2,9433.782954,4932.5,6151.7,8160.672003,6193.0,5379.561150,5717.3,64265.416106
3,4792.4,4884.3,8987.7,10351.184014,5189.0,6766.3,7298.042899,4948.4,5837.169846,5715.9,64770.396759
4,4920.9,2540.7,4403.6,9744.274392,1898.2,3904.0,4269.114270,2573.8,4678.933772,4469.3,43402.822435
5,0.0,0.0,0.0,52.757669,0.0,0.0,48.410989,0.0,35.740125,0.0,136.908783
6,4485.5,3908.4,4727.9,7667.195073,4055.1,5936.6,6424.930937,4196.2,4717.104260,4717.6,50836.530270
7,3959.2,3900.5,5093.2,7143.890392,3858.6,5663.8,6753.418571,4148.2,2983.066172,4580.9,48084.775136
8,4026.4,4408.7,4906.6,6633.815988,3669.4,4825.0,5955.959110,3652.1,3833.747943,4687.1,46598.823040
9,4101.7,3597.4,4792.2,7169.335130,3581.4,5235.2,6796.108664,4769.2,3788.950157,4612.4,48443.893951
